In [1]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import requests
import bs4

Before we build our dataframes, we'll create a few functions that gather the desired data. After creating each function, we'll test that it works on a single URL.

In [2]:
# gathering general data about the ski resort
def gatherGeneralData(url):
    """Takes a url containing data about a ski resort and returns a dataframe 
    with the data given in the html soup"""
    response = requests.get(url+"ski-resort")
    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    #Creating the dataframe
    columns = ["Location", "Resort", "Pass","Limited or unlimited", "Total Lifts", "Summit height", "Vertical Drop", "Base Elevation"]
    local_df = pd.DataFrame(columns=columns)

    databoxes = soup.find_all(class_="styles_box__1sXJN")

    #Gathering data from "Terrain Overview"
    terrain_data = databoxes[1]
    terrain_data_cols = terrain_data.find_all(class_="styles_secondary__2Uu9u")
    terrain_data_values = terrain_data.find_all(class_="styles_value__fB0LV")
    for i in range(0,len(terrain_data_cols)):
        local_df[terrain_data_cols[i].text] = [terrain_data_values[i].text]

    #Getting total lifts
    local_df["Total Lifts"] = soup.find(class_="styles_totalLifts__3mtxO").find(
                                            class_="styles_title__iYnrH").text.lstrip("Total Lifts: ")

    #Getting elevation info
    elevation_data = databoxes[3].find_all(class_="styles_distance__3eFw3")
    local_df["Summit height"] = elevation_data[0].text
    local_df["Vertical Drop"] = elevation_data[1].text
    local_df["Base Elevation"] = elevation_data[2].text


    #Gathering from "important dates"
    # box = soup.find_all(class_="styles_box__1sXJN")[4]
    important_dates = databoxes[4]
    im_d_data = important_dates.find_all(class_="styles_item__31bBH")
    for i in range(0, len(im_d_data)):
        local_df[(im_d_data[i].find("div").text)] = im_d_data[i].find_all("span")[0].text.replace(",", " ")

    #Getting Resort name and location
    local_df["Resort"] = soup.find(class_="styles_resort__1wghR").text
    local_df["Location"] = soup.find(class_="styles_region__2x2a_").text

    return local_df

url = "https://www.onthesnow.com/colorado/beaver-creek/"
gatherGeneralData(url)


,Location,Resort,Pass,Limited or unlimited,Total Lifts,Summit height,Vertical Drop,Base Elevation,Beginners Runs,Intermediate Runs,...,Runs in Total,Longest Run,Skiable Terrain,Snow Making,Projected Opening,Projected Closing,Projected Days Open,Days Open Last Year,Years Open,Average Snowfall
0,colorado,Beaver Creek,NaN,NaN,24,11440',3340',8100',38%,30%,...,150,2.8 mi,1832 ac,650 ac,November 29 20212021/11/29,April 10 20222022/04/10,141,144,42,"323"""


In [3]:
# gathering monthly snowfall data from the current season
def gatherMonthlyData(url):
    """Takes a url containing data about a ski resort and returns a dataframe 
    with the data given in the html soup"""

    #Getting the HTML
    response = requests.get(url+"historical-snowfall")
    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    #Gathering both monthly and annual data
    tables = soup.find_all(class_ = "styles_table__367yU")
    month_table = tables[0]

    #Creating table to store monthly data:
    columns = month_table.tr.find_all("span")
    columns_list = ["Location", "Resort", "Pass", "Month"]
    for col in columns:
        columns_list.append(col.text)

    local_df = pd.DataFrame(columns = columns_list) #dataframe to store the final data

    #Creating lists to store the data for the columns
    months = ["October", "November", "December", "January", "February", "March", "April", "May"]
    avg_snowfall = []
    snowfall_days = []
    avg_base_depth = []
    avg_sum_depth = []
    max_base_depth = []
    biggest_snowfall = []


    #extracting the data from the columns
    rows = month_table.tbody.find_all("tr")
    for row in range(0, len(rows)):
        rowdata = rows[row].find_all("td")
        avg_snowfall.append(rowdata[0].text)
        snowfall_days.append(rowdata[1].text)
        avg_base_depth.append(rowdata[2].text)
        avg_sum_depth.append(rowdata[3].text)
        max_base_depth.append(rowdata[4].text)
        biggest_snowfall.append(rowdata[5].text)

    #populating the table with the data from the lists
    local_df["Month"] = pd.Series(months)
    local_df.iloc[:, 4] = pd.Series(avg_snowfall)
    local_df.iloc[:, 5] = pd.Series(snowfall_days)
    local_df.iloc[:, 6] = pd.Series(avg_base_depth)
    local_df.iloc[:, 7] = pd.Series(avg_sum_depth)
    local_df.iloc[:, 8] = pd.Series(max_base_depth)
    local_df.iloc[:, 9] = pd.Series(biggest_snowfall)
    local_df["Resort"].fillna(soup.find(class_="styles_resort__1wghR").text, inplace=True)
    local_df["Location"].fillna(soup.find(class_="styles_region__2x2a_").text, inplace=True)
    return local_df

gatherMonthlyData(url = "https://www.onthesnow.com/british-columbia/whistler-blackcomb/")

,Location,Resort,Pass,Month,Average Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,british-columbia,Whistler Blackcomb,NaN,October,"40""",8,"18""","17""","21""","9"""
1,british-columbia,Whistler Blackcomb,NaN,November,"77""",19,"34""","51""","45""","14"""
2,british-columbia,Whistler Blackcomb,NaN,December,"83""",16,"49""","79""","63""","19"""
3,british-columbia,Whistler Blackcomb,NaN,January,"45""",14,"71""","91""","79""","11"""
4,british-columbia,Whistler Blackcomb,NaN,February,"61""",15,"82""","104""","92""","14"""
5,british-columbia,Whistler Blackcomb,NaN,March,"23""",10,"74""","74""","82""","5"""
6,british-columbia,Whistler Blackcomb,NaN,April,"3""",1,"36""","43""","43""","2"""
7,british-columbia,Whistler Blackcomb,NaN,May,"0""",0,"0""","3""","0""","0"""


In [4]:
# gathering annual snowfall data
def gatherAnnualData(url):
    """Takes a url containing data about a ski resort and returns a dataframe 
    with the data given in the html soup"""

    #Getting the HTML
    response = requests.get(url+"historical-snowfall")
    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    #Gathering both monthly and annual data
    tables = soup.find_all(class_ = "styles_table__367yU")
    annual_table = tables[1]

    #Creating table to store annual data:
    columns = annual_table.tr.find_all("span")
    columns_list = ["Location", "Resort", "Pass", "Year"]
    for col in columns:
        columns_list.append(col.text)

    local_df = pd.DataFrame(columns = columns_list) #dataframe to store the final data

    #Creating lists to store the data for the columns
    years = []
    total_snow = []
    snow_days = []
    avg_base_dep = []
    avg_sum_dep = []
    max_base_dep = []
    biggest_snow = []

    #extracting the data from the columns
    rows = annual_table.tbody.findAll("tr")
    for row in range(0, len(rows) - 1): # We do -1 here to elimate the last row of averages
        years.append(rows[row].find("span").text)
        rowdata = rows[row].find_all("td")
        total_snow.append(rowdata[0].text)
        snow_days.append(rowdata[1].text)
        avg_base_dep.append(rowdata[2].text)
        avg_sum_dep.append(rowdata[3].text)
        max_base_dep.append(rowdata[4].text)
        biggest_snow.append(rowdata[5].text)

    #populating the table with the data from the lists
    local_df.iloc[:, 3] = pd.Series(years)
    local_df.iloc[:, 4] = pd.Series(total_snow)
    local_df.iloc[:, 5] = pd.Series(snow_days)
    local_df.iloc[:, 6] = pd.Series(avg_base_dep)
    local_df.iloc[:, 7] = pd.Series(avg_sum_dep)
    local_df.iloc[:, 8] = pd.Series(max_base_dep)
    local_df.iloc[:, 9] = pd.Series(biggest_snow)
    local_df["Resort"].fillna(soup.find(class_="styles_resort__1wghR").text, inplace=True)
    local_df["Location"].fillna(soup.find(class_="styles_region__2x2a_").text, inplace=True)
    return local_df

gatherAnnualData(url = "https://www.onthesnow.com/british-columbia/whistler-blackcomb/")

,Location,Resort,Pass,Year,Total Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,british-columbia,Whistler Blackcomb,NaN,2012 - 2013,"280""",61,"52""","65""","112""","19"""
1,british-columbia,Whistler Blackcomb,NaN,2013 - 2014,"274""",80,"54""","53""","117""","25"""
2,british-columbia,Whistler Blackcomb,NaN,2014 - 2015,"219""",68,"31""","40""","79""","13"""
3,british-columbia,Whistler Blackcomb,NaN,2015 - 2016,"435""",98,"68""","67""","148""","21"""
4,british-columbia,Whistler Blackcomb,NaN,2016 - 2017,"505""",110,"27""","77""","106""","25"""
5,british-columbia,Whistler Blackcomb,NaN,2017 - 2018,"316""",94,"81""","70""","132""","24"""
6,british-columbia,Whistler Blackcomb,NaN,2018 - 2019,"374""",82,"44""","66""","106""","29"""
7,british-columbia,Whistler Blackcomb,NaN,2019 - 2020,"295""",76,"40""","39""","108""","18"""
8,british-columbia,Whistler Blackcomb,NaN,2020 - 2021,"281""",76,"45""","41""","118""","25"""
9,british-columbia,Whistler Blackcomb,NaN,2021 - To Date,"321""",76,"0""","69""","0""","24"""


Next, we'll create a function that gets all the urls for the data from a text file. I created two text files containing all the URL's for the resorts and split them into Epic and Ikon files. All of the resorts listed on the https://www.onthesnow.com/epic-pass/skireport and https://www.onthesnow.com/ikon-pass/skireport sites are included except resorts outside the northern hemisphere (therefore no Australian resorts were included).

In [5]:
def geturls(file):
    urls_f = open(file, "r")
    urls = []
    for line in urls_f:
        urls.append(line.rstrip("skireport\n"))

    return urls

Time to gather the data for and build the dataframes.

In [6]:
#Getting the urls of all of the ikon resorts
ikon_urls = geturls("ikonurls.txt")

#Creating a dataframe out of the first url to append all the other data to
ikonGenInfoDf = gatherGeneralData(ikon_urls[0])
ikonMonthlyDf = gatherMonthlyData(ikon_urls[0])
ikonAnnualDf = gatherAnnualData(ikon_urls[0])


#Extracting the data from the rest of the urls
for i in range(1, len(ikon_urls)):
    generaldf = gatherGeneralData(ikon_urls[i])
    monthlydf = gatherMonthlyData(ikon_urls[i])
    annualdf = gatherAnnualData(ikon_urls[i])

    ikonGenInfoDf = ikonGenInfoDf.append(generaldf, ignore_index=True)
    ikonMonthlyDf = ikonMonthlyDf.append(monthlydf, ignore_index=True)
    ikonAnnualDf = ikonAnnualDf.append(annualdf, ignore_index=True)

ikonGenInfoDf["Pass"] = "Ikon"
ikonMonthlyDf["Pass"] = "Ikon"
ikonAnnualDf["Pass"] = "Ikon"

    

In [7]:
display(ikonGenInfoDf.head(10))
ikonGenInfoDf.info()

,Location,Resort,Pass,Limited or unlimited,Total Lifts,Summit height,Vertical Drop,Base Elevation,Runs in Total,Longest Run,...,Projected Closing,Projected Days Open,Days Open Last Year,Years Open,Average Snowfall,Beginners Runs,Intermediate Runs,Advanced Runs,Expert Runs,Night Skiing
0,utah,Alta,Ikon,NaN,6,11068',2538',8530',119,1.3 mi,...,April 25 20222022/04/25,142,156,84,"547""",NaN,NaN,NaN,NaN,NaN
1,colorado,Arapahoe Basin,Ikon,NaN,9,13050',2530',10780',147,1.5 mi,...,June 05 20222022/06/05,270,234,76,"350""",7%,20%,49%,24%,NaN
2,colorado,Aspen Snowmass,Ikon,NaN,40,12510',4406',8104',336,5.3 mi,...,April 17 20222022/04/17,148,150,75,"300""",NaN,NaN,NaN,NaN,NaN
3,california,Bear Mountain,Ikon,NaN,12,8805',1665',7140',15,1.5 mi,...,April 16 20222022/04/16,135,122,79,"100""",20%,33%,33%,13%,NaN
4,montana,Big Sky,Ikon,NaN,36,11166',4350',7500',317,6 mi,...,April 24 20222022/04/24,143,144,49,"400""",15%,25%,42%,18%,NaN
5,ontario,Blue Mountain,Ikon,NaN,11,1476',722',755',55,1 mi,...,April 03 20222022/04/03,149,135,81,"110""",22%,33%,16%,29%,260 ac
6,michigan,Boyne Mountain,Ikon,NaN,12,1120',500',620',60,1 mi,...,April 24 20222022/04/24,130,122,74,"140""",49%,28%,20%,3%,200 ac
7,utah,Brighton,Ikon,NaN,7,10500',1745',8755',66,1.2 mi,...,April 20 20222022/04/20,152,138,86,"500""",NaN,NaN,NaN,NaN,200 ac
8,colorado,Copper Mountain,Ikon,NaN,24,12313',2738',9712',150,1.7 mi,...,April 24 20222022/04/24,163,153,50,"300""",25%,24%,34%,17%,NaN
9,washington,Crystal Mountain,Ikon,NaN,11,7012',3100',4400',86,2.5 mi,...,May 08 20222022/05/08,142,N.A.,60,"486""",8%,31%,32%,29%,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              41 non-null     object
 1   Resort                41 non-null     object
 2   Pass                  41 non-null     object
 3   Limited or unlimited  0 non-null      object
 4   Total Lifts           41 non-null     object
 5   Summit height         41 non-null     object
 6   Vertical Drop         41 non-null     object
 7   Base Elevation        41 non-null     object
 8   Runs in Total         41 non-null     object
 9   Longest Run           41 non-null     object
 10  Skiable Terrain       41 non-null     object
 11  Snow Making           38 non-null     object
 12  Projected Opening     41 non-null     object
 13  Projected Closing     41 non-null     object
 14  Projected Days Open   41 non-null     object
 15  Days Open Last Year   41 non-null     obje

In [8]:
display(ikonMonthlyDf.head(15))
ikonMonthlyDf.info()

,Location,Resort,Pass,Month,Average Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,utah,Alta,Ikon,October,"11""",3,"16""","20""","20""","5"""
1,utah,Alta,Ikon,November,"82""",14,"45""","45""","65""","16"""
2,utah,Alta,Ikon,December,"76""",13,"69""","71""","87""","16"""
3,utah,Alta,Ikon,January,"73""",13,"92""","93""","107""","15"""
4,utah,Alta,Ikon,February,"61""",11,"106""","106""","118""","14"""
5,utah,Alta,Ikon,March,"37""",7,"97""","91""","108""","12"""
6,utah,Alta,Ikon,April,"0""",0,"22""","14""","23""","0"""
7,utah,Alta,Ikon,May,"0""",0,"0""","0""","0""","0"""
8,colorado,Arapahoe Basin,Ikon,October,"33""",10,"20""","20""","25""","8"""
9,colorado,Arapahoe Basin,Ikon,November,"43""",13,"29""","29""","41""","10"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              328 non-null    object
 1   Resort                328 non-null    object
 2   Pass                  328 non-null    object
 3   Month                 328 non-null    object
 4   Average Snowfall      328 non-null    object
 5   Snowfall Days         328 non-null    object
 6   Average Base Depth    328 non-null    object
 7   Average Summit Depth  328 non-null    object
 8   Max Base Depth        328 non-null    object
 9   Biggest Snowfall      328 non-null    object
dtypes: object(10)
memory usage: 25.8+ KB


In [9]:
display(ikonAnnualDf.head(15))
ikonAnnualDf.info()

,Location,Resort,Pass,Year,Total Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,utah,Alta,Ikon,2012 - 2013,"366""",76,"50""","52""","113""","17"""
1,utah,Alta,Ikon,2013 - 2014,"353""",68,"64""","60""","124""","14"""
2,utah,Alta,Ikon,2014 - 2015,"278""",56,"44""","42""","90""","19"""
3,utah,Alta,Ikon,2015 - 2016,"286""",60,"51""","51""","116""","19"""
4,utah,Alta,Ikon,2016 - 2017,"304""",55,"64""","63""","145""","19"""
5,utah,Alta,Ikon,2017 - 2018,"190""",42,"54""","51""","115""","15"""
6,utah,Alta,Ikon,2018 - 2019,"656""",92,"74""","73""","180""","27"""
7,utah,Alta,Ikon,2019 - 2020,"326""",53,"50""","49""","134""","24"""
8,utah,Alta,Ikon,2020 - 2021,"330""",52,"48""","50""","116""","27"""
9,utah,Alta,Ikon,2021 - To Date,"265""",46,"72""","71""","114""","24"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              410 non-null    object
 1   Resort                410 non-null    object
 2   Pass                  410 non-null    object
 3   Year                  410 non-null    object
 4   Total Snowfall        410 non-null    object
 5   Snowfall Days         410 non-null    object
 6   Average Base Depth    410 non-null    object
 7   Average Summit Depth  410 non-null    object
 8   Max Base Depth        410 non-null    object
 9   Biggest Snowfall      410 non-null    object
dtypes: object(10)
memory usage: 32.2+ KB


In [10]:
#Getting the urls of all of the epic resorts
Epic_urls = geturls("epicurls.txt")

#Creating a dataframe out of the first url to append all the other data to
epicGenInfoDf = gatherGeneralData(Epic_urls[0])
epicMonthlyDf = gatherMonthlyData(Epic_urls[0])
epicAnnualDf = gatherAnnualData(Epic_urls[0])


#Extracting the data from the rest of the urls
for i in range(1, len(Epic_urls)):
    generaldf = gatherGeneralData(Epic_urls[i])
    monthlydf = gatherMonthlyData(Epic_urls[i])
    annualdf = gatherAnnualData(Epic_urls[i])

    epicGenInfoDf = epicGenInfoDf.append(generaldf, ignore_index=True)
    epicMonthlyDf = epicMonthlyDf.append(monthlydf, ignore_index=True)
    epicAnnualDf = epicAnnualDf.append(annualdf, ignore_index=True)

epicGenInfoDf["Pass"] = "Epic"
epicMonthlyDf["Pass"] = "Epic"
epicAnnualDf["Pass"] = "Epic"


In [11]:
display(epicGenInfoDf.head(10))
display(epicGenInfoDf.info())
display(epicMonthlyDf.head(15))
display(epicMonthlyDf.info())
display(epicAnnualDf.head(15))
display(epicAnnualDf.info())

,Location,Resort,Pass,Limited or unlimited,Total Lifts,Summit height,Vertical Drop,Base Elevation,Beginners Runs,Intermediate Runs,...,Longest Run,Skiable Terrain,Snow Making,Projected Opening,Projected Closing,Projected Days Open,Days Open Last Year,Years Open,Average Snowfall,Night Skiing
0,colorado,Beaver Creek,Epic,NaN,24,11440',3340',8100',38%,30%,...,2.8 mi,1832 ac,650 ac,November 29 20212021/11/29,April 10 20222022/04/10,141,144,42,"323""",NaN
1,colorado,Breckenridge,Epic,NaN,34,12998',3398',9600',13%,23%,...,3.5 mi,2908 ac,600 ac,November 12 20212021/11/12,May 30 20222022/05/30,199,204,61,"300""",NaN
2,northern-alps,Brides les Bains,Epic,NaN,41,2950m,2350m,600m,9%,40%,...,1.1 km,150 km,75 km,December 04 20212021/12/04,April 15 20222022/04/15,120,120,30,0cm,NaN
3,northern-alps,Courchevel,Epic,NaN,55,2738m,1388m,1100m,19%,38%,...,9 km,150 km,87 km,December 04 20212021/12/04,April 22 20222022/04/22,140,140,76,800cm,3 km
4,british-columbia,Fernie,Epic,NaN,10,7001',3550',3451',NaN,NaN,...,3.1 mi,2504 ac,125 ac,December 11 20212021/12/11,April 10 20222022/04/10,135,130,60,"444""",NaN
5,trentino,Folgàrida - Marilléva,Epic,NaN,25,2040m,770m,1270m,NaN,32%,...,NaN,63 km,NaN,December 04 20212021/12/04,April 18 20222022/04/18,N.A.,N.A.,N.A.,0cm,NaN
6,california,Heavenly,Epic,NaN,28,10067',3500',7170',8%,62%,...,5.5 mi,4800 ac,3379 ac,December 11 20212021/12/11,April 24 20222022/04/24,144,155,67,"360""",NaN
7,colorado,Keystone,Epic,NaN,20,12408',3128',9280',16%,43%,...,3.5 mi,3148 ac,662 ac,October 22 20212021/10/22,April 10 20222022/04/10,183,151,52,"235""",288 ac
8,british-columbia,Kicking Horse,Epic,NaN,5,8038',4134',3904',NaN,NaN,...,6.2 mi,2800 ac,516 ac,December 10 20212021/12/10,April 10 20222022/04/10,128,120,22,"288""",NaN
9,british-columbia,Kimberley,Epic,NaN,5,6503',2464',4035',NaN,NaN,...,4 mi,1800 ac,120 ac,December 18 20212021/12/18,April 10 20222022/04/10,137,109,64,"150""",120 ac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              46 non-null     object
 1   Resort                46 non-null     object
 2   Pass                  46 non-null     object
 3   Limited or unlimited  0 non-null      object
 4   Total Lifts           46 non-null     object
 5   Summit height         46 non-null     object
 6   Vertical Drop         46 non-null     object
 7   Base Elevation        46 non-null     object
 8   Beginners Runs        29 non-null     object
 9   Intermediate Runs     41 non-null     object
 10  Advanced Runs         41 non-null     object
 11  Expert Runs           36 non-null     object
 12  Runs in Total         46 non-null     object
 13  Longest Run           38 non-null     object
 14  Skiable Terrain       46 non-null     object
 15  Snow Making           40 non-null     obje

None

,Location,Resort,Pass,Month,Average Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,colorado,Beaver Creek,Epic,October,"20""",5,"19""","15""","22""","6"""
1,colorado,Beaver Creek,Epic,November,"50""",14,"26""","26""","35""","9"""
2,colorado,Beaver Creek,Epic,December,"46""",12,"38""","38""","47""","10"""
3,colorado,Beaver Creek,Epic,January,"47""",14,"49""","49""","57""","9"""
4,colorado,Beaver Creek,Epic,February,"50""",11,"58""","58""","69""","10"""
5,colorado,Beaver Creek,Epic,March,"15""",4,"53""","48""","60""","7"""
6,colorado,Beaver Creek,Epic,April,"0""",0,"7""","7""","7""","0"""
7,colorado,Beaver Creek,Epic,May,"0""",0,"0""","0""","0""","0"""
8,colorado,Breckenridge,Epic,October,"30""",8,"22""","21""","26""","8"""
9,colorado,Breckenridge,Epic,November,"59""",15,"32""","31""","67""","13"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              368 non-null    object
 1   Resort                368 non-null    object
 2   Pass                  368 non-null    object
 3   Month                 368 non-null    object
 4   Average Snowfall      308 non-null    object
 5   Snowfall Days         308 non-null    object
 6   Average Base Depth    308 non-null    object
 7   Average Summit Depth  308 non-null    object
 8   Max Base Depth        308 non-null    object
 9   Biggest Snowfall      308 non-null    object
dtypes: object(10)
memory usage: 28.9+ KB


None

,Location,Resort,Pass,Year,Total Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,colorado,Beaver Creek,Epic,2012 - 2013,"204""",62,"28""","26""","75""","11"""
1,colorado,Beaver Creek,Epic,2013 - 2014,"227""",61,"38""","37""","86""","23"""
2,colorado,Beaver Creek,Epic,2014 - 2015,"210""",49,"29""","28""","68""","16"""
3,colorado,Beaver Creek,Epic,2015 - 2016,"298""",75,"39""","37""","70""","13"""
4,colorado,Beaver Creek,Epic,2016 - 2017,"247""",64,"32""","31""","69""","14"""
5,colorado,Beaver Creek,Epic,2017 - 2018,"193""",43,"25""","25""","68""","12"""
6,colorado,Beaver Creek,Epic,2018 - 2019,"276""",73,"36""","36""","94""","16"""
7,colorado,Beaver Creek,Epic,2019 - 2020,"216""",57,"26""","24""","65""","9"""
8,colorado,Beaver Creek,Epic,2020 - 2021,"202""",61,"24""","24""","56""","11"""
9,colorado,Beaver Creek,Epic,2021 - To Date,"185""",53,"44""","44""","67""","10"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              460 non-null    object
 1   Resort                460 non-null    object
 2   Pass                  460 non-null    object
 3   Year                  460 non-null    object
 4   Total Snowfall        460 non-null    object
 5   Snowfall Days         460 non-null    object
 6   Average Base Depth    460 non-null    object
 7   Average Summit Depth  460 non-null    object
 8   Max Base Depth        460 non-null    object
 9   Biggest Snowfall      460 non-null    object
dtypes: object(10)
memory usage: 36.1+ KB


None

In [12]:
#Creating final dataframes
geninfoDf = ikonGenInfoDf.append(epicGenInfoDf, ignore_index=True)
annualdf = ikonAnnualDf.append(epicAnnualDf, ignore_index=True)
monthlydf = ikonMonthlyDf.append(epicMonthlyDf, ignore_index=True)

display(geninfoDf.head(10))
display(annualdf.head(10))
display(monthlydf.head(10))

,Location,Resort,Pass,Limited or unlimited,Total Lifts,Summit height,Vertical Drop,Base Elevation,Runs in Total,Longest Run,...,Projected Closing,Projected Days Open,Days Open Last Year,Years Open,Average Snowfall,Beginners Runs,Intermediate Runs,Advanced Runs,Expert Runs,Night Skiing
0,utah,Alta,Ikon,NaN,6,11068',2538',8530',119,1.3 mi,...,April 25 20222022/04/25,142,156,84,"547""",NaN,NaN,NaN,NaN,NaN
1,colorado,Arapahoe Basin,Ikon,NaN,9,13050',2530',10780',147,1.5 mi,...,June 05 20222022/06/05,270,234,76,"350""",7%,20%,49%,24%,NaN
2,colorado,Aspen Snowmass,Ikon,NaN,40,12510',4406',8104',336,5.3 mi,...,April 17 20222022/04/17,148,150,75,"300""",NaN,NaN,NaN,NaN,NaN
3,california,Bear Mountain,Ikon,NaN,12,8805',1665',7140',15,1.5 mi,...,April 16 20222022/04/16,135,122,79,"100""",20%,33%,33%,13%,NaN
4,montana,Big Sky,Ikon,NaN,36,11166',4350',7500',317,6 mi,...,April 24 20222022/04/24,143,144,49,"400""",15%,25%,42%,18%,NaN
5,ontario,Blue Mountain,Ikon,NaN,11,1476',722',755',55,1 mi,...,April 03 20222022/04/03,149,135,81,"110""",22%,33%,16%,29%,260 ac
6,michigan,Boyne Mountain,Ikon,NaN,12,1120',500',620',60,1 mi,...,April 24 20222022/04/24,130,122,74,"140""",49%,28%,20%,3%,200 ac
7,utah,Brighton,Ikon,NaN,7,10500',1745',8755',66,1.2 mi,...,April 20 20222022/04/20,152,138,86,"500""",NaN,NaN,NaN,NaN,200 ac
8,colorado,Copper Mountain,Ikon,NaN,24,12313',2738',9712',150,1.7 mi,...,April 24 20222022/04/24,163,153,50,"300""",25%,24%,34%,17%,NaN
9,washington,Crystal Mountain,Ikon,NaN,11,7012',3100',4400',86,2.5 mi,...,May 08 20222022/05/08,142,N.A.,60,"486""",8%,31%,32%,29%,NaN


,Location,Resort,Pass,Year,Total Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,utah,Alta,Ikon,2012 - 2013,"366""",76,"50""","52""","113""","17"""
1,utah,Alta,Ikon,2013 - 2014,"353""",68,"64""","60""","124""","14"""
2,utah,Alta,Ikon,2014 - 2015,"278""",56,"44""","42""","90""","19"""
3,utah,Alta,Ikon,2015 - 2016,"286""",60,"51""","51""","116""","19"""
4,utah,Alta,Ikon,2016 - 2017,"304""",55,"64""","63""","145""","19"""
5,utah,Alta,Ikon,2017 - 2018,"190""",42,"54""","51""","115""","15"""
6,utah,Alta,Ikon,2018 - 2019,"656""",92,"74""","73""","180""","27"""
7,utah,Alta,Ikon,2019 - 2020,"326""",53,"50""","49""","134""","24"""
8,utah,Alta,Ikon,2020 - 2021,"330""",52,"48""","50""","116""","27"""
9,utah,Alta,Ikon,2021 - To Date,"265""",46,"72""","71""","114""","24"""


,Location,Resort,Pass,Month,Average Snowfall,Snowfall Days,Average Base Depth,Average Summit Depth,Max Base Depth,Biggest Snowfall
0,utah,Alta,Ikon,October,"11""",3,"16""","20""","20""","5"""
1,utah,Alta,Ikon,November,"82""",14,"45""","45""","65""","16"""
2,utah,Alta,Ikon,December,"76""",13,"69""","71""","87""","16"""
3,utah,Alta,Ikon,January,"73""",13,"92""","93""","107""","15"""
4,utah,Alta,Ikon,February,"61""",11,"106""","106""","118""","14"""
5,utah,Alta,Ikon,March,"37""",7,"97""","91""","108""","12"""
6,utah,Alta,Ikon,April,"0""",0,"22""","14""","23""","0"""
7,utah,Alta,Ikon,May,"0""",0,"0""","0""","0""","0"""
8,colorado,Arapahoe Basin,Ikon,October,"33""",10,"20""","20""","25""","8"""
9,colorado,Arapahoe Basin,Ikon,November,"43""",13,"29""","29""","41""","10"""


I opted to do some additional scraping because the OnTheSnow site did not include information on whether the access offered to these resorts by the Epic and Ikon passes was unlimited or limited (limited access meaning skiers can only ski X amount of days, while unlimited gives the skier unlimited access to skiing at the resort.) In the following cells I scrape https://epicorikon.com/ to create lists of all the resorts offered by both passes that are either limited or unlimited in access, and create a dictionary to map the names of the resorts in our database to the limited or unlimited labels.

In [13]:
#Creating maps for "limited" and "unlimited" resorts
url = "https://epicorikon.com/"
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
epic = soup.find(class_="sc-1asrjzk-0 sc-1asrjzk-1 iqwWpU epic")
ikon = soup.find(class_="sc-1asrjzk-0 sc-1asrjzk-1 iWlgOA ikon")

epic_unlimited = epic.find(class_="full-resort-list").find_all(class_="sc-1601ldf-0 cdwFek")
epic_limited = epic.find(class_="limited-resort-list").find_all(class_="sc-1601ldf-0 cdwFek")
ikon_unlimited = ikon.find(class_="full-resort-list").find_all(class_="sc-1601ldf-0 cdwFek")
ikon_limited = ikon.find(class_="limited-resort-list").find_all(class_="sc-1601ldf-0 cdwFek")

In [14]:
def createdict(resorts, limit):
    dict = {}
    for resort in resorts:
        name = resort.text.split(',')[0].split(" ")[0]
        dict.update({name:limit})

    return dict

dict = createdict(epic_limited, "limited")
dict.update(createdict(epic_unlimited, "unlimited"))
dict.update(createdict(ikon_unlimited, "unlimited"))
dict.update(createdict(ikon_limited, "limited"))
dict.update({"Summit at Snoqualmie":"limited"})
dict

{'Telluride': 'limited',
 'Fernie': 'limited',
 'Kicking': 'limited',
 'Kimberley': 'limited',
 'Nakiska': 'limited',
 'Mont-Sainte': 'limited',
 'Stoneham': 'limited',
 'Hakuba47': 'limited',
 'Hakuba': 'limited',
 'Jiigatake': 'limited',
 'Kashimayari': 'limited',
 'Tsugaike': 'limited',
 'Rusutsu': 'limited',
 'Skirama': 'limited',
 'Les': 'limited',
 'Arlberg': 'limited',
 '4': 'limited',
 'Vail': 'unlimited',
 'Beaver': 'unlimited',
 'Whistler': 'unlimited',
 'Breckenridge': 'unlimited',
 'Keystone': 'unlimited',
 'Crested': 'unlimited',
 'Park': 'unlimited',
 'Heavenly': 'unlimited',
 'Northstar': 'unlimited',
 'Kirkwood': 'unlimited',
 'Stevens': 'unlimited',
 'Stowe': 'unlimited',
 'Okemo': 'unlimited',
 'Mount': 'unlimited',
 'Wildcat': 'unlimited',
 'Attitash': 'unlimited',
 'Crotched': 'unlimited',
 'Hunter': 'unlimited',
 'Liberty': 'unlimited',
 'Roundtop': 'unlimited',
 'Whitetail': 'unlimited',
 'Jack': 'unlimited',
 'Big': 'limited',
 'Laurel': 'unlimited',
 'Seven': 'u

In [15]:
for resort in dict:
    i = geninfoDf[geninfoDf["Resort"].str.contains(resort)]["Limited or unlimited"].index
    geninfoDf["Limited or unlimited"][i] = dict.get(resort)

geninfoDf["Limited or unlimited"].fillna("limited", inplace=True)


Something to note is that Snowbasin and Sun Valley are given limited access by both passes, that's why they show up twice in the geninfo database.

In [16]:
geninfoDf["Resort"].value_counts()

Snowbasin          2
Sun Valley         2
Alta               1
Meribel            1
Orelle             1
                  ..
Steamboat          1
Solitude           1
Snowbird           1
Snow Summit        1
Wilmot Mountain    1
Name: Resort, Length: 85, dtype: int64

All there is to do now is create csv's out of the data.

In [17]:
geninfoDf.to_csv("skiresorts.csv", index=False)
annualdf.to_csv("annualsnow.csv", index=False)
monthlydf.to_csv("currentseason.csv", index=False)

In [18]:
geninfoDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              87 non-null     object
 1   Resort                87 non-null     object
 2   Pass                  87 non-null     object
 3   Limited or unlimited  87 non-null     object
 4   Total Lifts           87 non-null     object
 5   Summit height         87 non-null     object
 6   Vertical Drop         87 non-null     object
 7   Base Elevation        87 non-null     object
 8   Runs in Total         87 non-null     object
 9   Longest Run           79 non-null     object
 10  Skiable Terrain       87 non-null     object
 11  Snow Making           78 non-null     object
 12  Projected Opening     87 non-null     object
 13  Projected Closing     87 non-null     object
 14  Projected Days Open   87 non-null     object
 15  Days Open Last Year   87 non-null     obje